In [29]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as pyplot
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn import svm
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import confusion_matrix,classification_report
from sklearn.preprocessing import StandardScaler,LabelEncoder
from sklearn.model_selection import train_test_split
%matplotlib inline
# 1,loading data,check null value
player=pd.read_csv('2010-11.csv',sep=',')
player.info()
player.isnull().sum()
# 2, processing data,set up unique labels and group names,use labelEncoder to set up the ratio to 0,1,
# use sns to shrew it into  a graph
# may play with the bins and make the bins lean more towards the high win ratio
bins=(-0.1,0.5,1.0)
group_names=['lose','win']
player['W_PCT']=pd.cut(player['W_PCT'], bins=bins,labels=group_names)
player['W_PCT'].unique()
label_ratio=LabelEncoder()
player['W_PCT']=label_ratio.fit_transform(player['W_PCT'].astype(str))
player.head(20)
player['W_PCT'].value_counts()
sns.countplot(x=player['W_PCT'])


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 452 entries, 0 to 451
Data columns (total 27 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   AGE              452 non-null    int64  
 1   W_PCT            452 non-null    float64
 2   MIN              452 non-null    float64
 3   FGM              452 non-null    float64
 4   FGA              452 non-null    float64
 5   FG_PCT           452 non-null    float64
 6   FG3M             452 non-null    float64
 7   FG3A             452 non-null    float64
 8   FG3_PCT          452 non-null    float64
 9   FTM              452 non-null    float64
 10  FTA              452 non-null    float64
 11  FT_PCT           452 non-null    float64
 12  OREB             452 non-null    float64
 13  DREB             452 non-null    float64
 14  REB              452 non-null    float64
 15  AST              452 non-null    float64
 16  TOV              452 non-null    float64
 17  STL             

,AGE,W_PCT,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,...,STL,BLK,BLKA,PF,PFD,PTS,PLUS_MINUS,NBA_FANTASY_PTS,DD2,TD3
0,26,0,21.8,3.7,9.9,0.375,1.2,4.0,0.297,2.1,...,0.6,0.1,0.7,1.9,2.1,10.7,-2.8,18.5,2,0
1,31,1,19.5,0.0,1.5,0.000,0.0,1.5,0.000,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,3.0,1.5,0,0
2,35,1,19.5,0.0,0.0,0.000,0.0,0.0,0.000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,-5.5,0.0,0,0
3,23,1,19.5,0.5,1.5,0.333,0.0,0.0,0.000,0.0,...,0.0,0.0,0.0,1.0,0.0,1.0,4.0,1.8,0,0
4,22,1,19.5,0.0,0.0,0.000,0.0,0.0,0.000,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,-2.0,0.0,0,0
5,22,1,19.5,0.0,0.0,0.000,0.0,0.0,0.000,0.0,...,0.0,0.0,0.0,2.0,0.0,0.0,-4.0,0.0,0,0
6,38,1,19.5,0.1,0.6,0.167,0.0,0.0,0.000,0.0,...,0.2,0.5,0.1,0.7,0.6,0.2,-2.7,4.0,0,0
7,35,1,19.5,0.2,1.0,0.188,0.0,0.0,0.000,0.1,...,0.0,0.3,0.1,0.8,0.3,0.5,-0.6,2.9,0,0
8,27,1,19.5,0.1,0.4,0.333,0.0,0.0,0.000,0.0,...,0.0,0.0,0.0,1.6,0.0,0.3,-2.3,2.3,0,0
9,27,1,19.5,0.8,2.2,0.363,0.3,0.9,0.297,0.1,...,0.1,0.0,0.1,0.7,0.2,2.0,-0.9,4.4,0,0


In [18]:
# 3, seperate the dataset as response variable and feature variables, get ready for model
X=player.drop('W_PCT',axis=1)
y=player['W_PCT']

In [19]:
# 4,Use train_test_split(package) to split the data to  Train data and Test data, default test_size 25%
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

In [30]:
#5 ,Scale up the data Applying Standart scaling to get optimized result eg, big number overweight the impact of small number
sc=StandardScaler()
X_train=sc.fit_transform(X_train)
X_test=sc.transform(X_test)
# X_test[:10]


In [31]:
# RANDOM FOREST -Least amount of parts to fine-tune,
# used for a medium sized data set
# 1,create randomForest variable,2,fit the training data to it,3 predict
rfc=RandomForestClassifier(n_estimators=200)
rfc.fit(X_train,y_train)
pred_rfc=rfc.predict(X_test)

In [35]:
# X_test[:20]
# the test data print out have been sacled

array([[ 2.80000000e+01,  1.95000000e+01,  2.20000000e+00,
         5.80000000e+00,  3.86000000e-01,  1.00000000e-01,
         8.00000000e-01,  1.88000000e-01,  9.00000000e-01,
         1.20000000e+00,  7.29000000e-01,  4.00000000e-01,
         1.60000000e+00,  2.00000000e+00,  3.40000000e+00,
         1.50000000e+00,  9.00000000e-01,  2.00000000e-01,
         4.00000000e-01,  1.70000000e+00,  1.40000000e+00,
         5.40000000e+00, -3.00000000e-01,  1.46000000e+01,
        -6.39685288e-17, -1.23016402e-17],
       [ 2.50000000e+01,  1.95000000e+01,  2.20000000e+00,
         5.50000000e+00,  3.99000000e-01,  1.20000000e+00,
         3.30000000e+00,  3.59000000e-01,  8.00000000e-01,
         9.00000000e-01,  8.71000000e-01,  2.00000000e-01,
         1.90000000e+00,  2.10000000e+00,  2.50000000e+00,
         1.30000000e+00,  1.10000000e+00,  1.00000000e-01,
         2.00000000e-01,  2.30000000e+00,  1.10000000e+00,
         6.40000000e+00,  3.70000000e+00,  1.49000000e+01,
        -6.39

In [36]:
pred_rfc[:20]

array([1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0])

In [47]:
# check the performence of the model
print(classification_report(y_test,pred_rfc))
print(confusion_matrix(y_test,pred_rfc))
# the consusion matrix report shows a lot of miss lable of 'win'


              precision    recall  f1-score   support

           0       0.77      0.82      0.80        40
           1       0.85      0.80      0.83        51

    accuracy                           0.81        91
   macro avg       0.81      0.81      0.81        91
weighted avg       0.82      0.81      0.81        91

[[33  7]
 [10 41]]


In [42]:
# SVM classifier
clf=svm.SVC()
clf.fit(X_train,y_train)
pred_clf=clf.predict(X_test)


In [44]:
# check performence of model
print(classification_report(y_test,pred_clf))
print(confusion_matrix(y_test,pred_clf))

              precision    recall  f1-score   support

           0       0.44      1.00      0.61        40
           1       0.00      0.00      0.00        51

    accuracy                           0.44        91
   macro avg       0.22      0.50      0.31        91
weighted avg       0.19      0.44      0.27        91

[[40  0]
 [51  0]]


C:\Users\cxxme\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [46]:
# Neural Network classifier
# works well with huge amount of data
mlpc=MLPClassifier(hidden_layer_sizes=(11,11,11),max_iter=500)
mlpc.fit(X_train,y_train)
pred_mlpc=mlpc.predict(X_test)
print(classification_report(y_test,pred_mlpc))
print(confusion_matrix(y_test,pred_mlpc))
# A convergence point is a machine learning models localized optimal state. 
# It basically means that the variables within the model have the best posible values in order to
# predict a target feature based on another set of features. In MLP, these variables are the weights within each neuron. 
# Generally, when a data set doesn't represent a organized and discernable pattern, machine learning algorithms might not be able to find a convergence point

              precision    recall  f1-score   support

           0       0.44      1.00      0.61        40
           1       0.00      0.00      0.00        51

    accuracy                           0.44        91
   macro avg       0.22      0.50      0.31        91
weighted avg       0.19      0.44      0.27        91

[[40  0]
 [51  0]]


C:\Users\cxxme\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:582: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\cxxme\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [57]:
# last step, use the choosen model and given new feature data to predict 
Xnew=[[20,18,3.6,9.9,0.36,1.2,2,0.28,2,2,0.7,0.2,0,1,2,1,0.5,0.1,0.5,1.3,2,9,-2,15,0,0]]


In [58]:
Xnew=sc.transform(Xnew)
ynew=rfc.predict(Xnew)
ynew

array([1])